# wps_convert

#### wps_convert is a process that runs the [convert](https://rvic.readthedocs.io/en/latest/user-guide/conversion/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [ ]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools import copy_http_content

# Ensure we are in the working directory with access to the data
if os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../../../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps'
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [ ]:
# NBVAL_IGNORE_OUTPUT
osprey.convert?

#### We can use the docstring to ensure we provide the appropriate parameters.

In [ ]:
# run convert
convert_output = osprey.convert(
    convert_config = "tests/data/configs/convert_local.cfg"
)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [ ]:
convert_data = Dataset(make_tmp_copy(convert_output))

# Metadata of the output of RVIC convert module
convert_meatadata = {
    "title": convert_data.title,
    "year": convert_data.history.split()[5],
    "month": convert_data.history.split()[2],
    "day": convert_data.history.split()[3],
    "RvicPourPointsFile": convert_data.RvicPourPointsFile,
    "RvicUHFile": convert_data.RvicUHFile,
    "RvicFdrFile": convert_data.RvicFdrFile,
    "RvicDomainFile": convert_data.RvicDomainFile,
}

In [ ]:
convert_config = read_config("tests/data/configs/convert_local.cfg")

# Expected output metadata determined from the input configuration file
expected_metadata = {
    "title": "RVIC parameter file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d"),
    "RvicPourPointsFile": convert_config["UHS_FILES"]["STATION_FILE"].split("/")[-1],
    "RvicUHFile": "unknown",
    "RvicFdrFile": "unknown",
    "RvicDomainFile": convert_config["DOMAIN"]["FILE_NAME"].split("/")[-1],
}

assert convert_meatadata == expected_metadata